In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

import itertools

# “Common Interest”: no. of common articles between two users.


## Preprocessing

In [ ]:
postings_1 = pd.read_csv('data/Postings_01052019_15052019.csv', sep=';')
postings_2 = pd.read_csv('data/Postings_16052019_31052019.csv', sep=';')
postings = pd.concat([postings_1,postings_2])
postings['PostingCreatedAt'] = postings.PostingCreatedAt.astype('datetime64')
postings['ArticlePublishingDate'] = postings.ArticlePublishingDate.astype('datetime64')
postings['UserCreatedAt'] = postings.UserCreatedAt.astype('datetime64')
postings.head()

In [ ]:
len(postings)

In [ ]:
postings.UserCommunityName.nunique()

In [ ]:
df = postings[["ID_CommunityIdentity","ID_Article","PostingCreatedAt","ArticleChannel","ArticleRessortName"]]

In [ ]:
df_day_diff = df[(df.PostingCreatedAt >= '2019-05-01') & 
                   (df.PostingCreatedAt < '2019-05-02')]

In [ ]:
df_day_diff.loc[df_day_diff['ID_CommunityIdentity'] == 117313]

In [ ]:
df_day_diff.loc[df_day_diff['ID_CommunityIdentity'] == 571503]

In [ ]:
df_day_diff.ArticleChannel.nunique()

In [ ]:
df_day_diff.ID_CommunityIdentity.nunique()

In [ ]:
df_day_diff.ID_Article.nunique()

In [ ]:
edges.drop_duplicates(keep=False,inplace=True)

In [ ]:
edges

In [ ]:
print(edges.to_string())

In [ ]:



    
df_day = df[(df.PostingCreatedAt >= '2019-05-01') & 
                   (df.PostingCreatedAt < '2019-05-02')].merge(df, 
                    on = 'ID_Article', suffixes = ['_1', '_2'], how = 'inner')

    
    



In [ ]:
df_day

In [ ]:
df_day = df_day[["ID_CommunityIdentity_1","ID_Article","ID_CommunityIdentity_2", "ArticleChannel_1"]]

In [ ]:
df_day.drop_duplicates(keep=False,inplace=True)

In [ ]:
df_day

In [ ]:
def countCommon(row):
    channel1 = df_day_diff["ID_Article"][df_day_diff["ID_CommunityIdentity"]==row["source"]]
    channel2 = df_day_diff["ID_Article"][df_day_diff["ID_CommunityIdentity"]==row["target"]]
    return len(list(set(channel1).intersection(channel2)))



In [ ]:
# Generate the combinations of countries from original Dataframe
from itertools import combinations
comb = combinations(df["ID_CommunityIdentity"].unique(), 2)
out = pd.DataFrame(list(comb), columns=["source", "target"])

# Find out the sports in common between coutries
out["weight"] = out.apply(countCommonSports, axis = 1)

In [ ]:
edges = df_day[["ID_CommunityIdentity_1","ID_CommunityIdentity_2","ID_Article","ArticleChannel_1"]].groupby(
            ["ID_CommunityIdentity_1","ID_CommunityIdentity_2"]).count()

In [ ]:
edges
edges.reset_index(inplace = True)
edges.rename(columns = {'ID_CommunityIdentity_1': 'source', 'ID_CommunityIdentity_2': 'target',
                        'ID_Article': 'weight','ArticleChannel_1': 'ArticleChannel_1'}, inplace = True)
edges
 


In [ ]:
edges

In [ ]:
edges = edges[edges.weight > 1]
edges = edges[edges.source != edges.target]

In [ ]:
edges=edges.loc[pd.DataFrame(np.sort(edges[['source','target']],1),index=edges.index).drop_duplicates(keep='first').index]


In [ ]:
edges

In [ ]:
print(edges.to_string())

In [ ]:
edges.to_csv("data/vote_to_edges_day_1.csv", index=False)

In [ ]:
edges_day_1 = pd.read_csv("data/vote_to_edges_day_1.csv")

In [ ]:
G_day_1 = nx.from_pandas_edgelist(edges_day_1, edge_attr = 'weight', create_using=nx.DiGraph())
G_day_1_undirected = nx.from_pandas_edgelist(edges_day_1, edge_attr = 'weight', create_using=nx.Graph())

In [ ]:
df_per_day = []

for i in range(1,6):
    df_day = df[(df.PostingCreatedAt >= '2019-05-{}'.format(str(i).zfill(2))) & 
                   (df.PostingCreatedAt < '2019-05-{}'.format(str(i+1).zfill(2)))].merge(df, 
                    on = 'ID_Article', suffixes = ['_1', '_2'], how = 'inner')
    df_day = df_day[["ID_CommunityIdentity_1","ID_Article","ID_CommunityIdentity_2"]]

    df_day['day'] = i
    df_day = df_day[['ID_CommunityIdentity_1', 'ID_Article', 'day', 
                     'ID_CommunityIdentity_2']]
    df_day.drop_duplicates(keep=False,inplace=True)
     
    df_per_day.append(df_day)



In [ ]:
df_per_day

## Outputting the csv for source,target,weight 

In [ ]:
edges_days = pd.DataFrame(columns = ['source', 'target', 'weight', 'day'])

for i in range(5):
    #df_per_day[i]=df_per_day[i][["ID_CommunityIdentity_1","ID_CommunityIdentity_2","ID_Article","Day"]]
    df_per_day[i].drop_duplicates(keep=False,inplace=True)
    edges_day = df_per_day[i][["ID_CommunityIdentity_1","ID_CommunityIdentity_2","ID_Article","day"]].groupby(
            ["ID_CommunityIdentity_1","ID_CommunityIdentity_2","day"]).count()
    print("1")
    edges_day.reset_index(inplace = True)
    print("2")
    
    edges_day.rename(columns = {'ID_CommunityIdentity_1': 'source', 'ID_CommunityIdentity_2': 'target',
                        'ID_Article': 'weight'}, inplace = True)
    #print(edges_day)
    print("3")
    #edges_day.drop_duplicates(keep=False,inplace=True)
    edges_day=edges_day.loc[pd.DataFrame(np.sort(edges_day[['source','target']],1),index=edges_day.index).drop_duplicates(keep='first').index]
    edges_day = edges_day[edges_day.weight > 1]
    edges_day = edges_day[edges_day.source != edges_day.target]
    #edges_day = edges_day[edges_day.source != edges_day.target]
    #edges_day['day'] = df_per_day[i].loc[0, 'Day']
    print("4")
    print(edges_day)
    edges_day.to_csv('edges_days.csv', mode='a',index=False)
    #edges_days.iloc[len(edges_days.index):,:] = edges_day
    #edges_days = pd.concat([edges_days, edges_day])
    print("5")
    print("------------")

In [ ]:
edges_days = pd.read_csv("edges_days.csv")



In [ ]:
edges_days

In [ ]:
edges_days.describe()

In [ ]:
dataTypeSeries = edges_days.dtypes
print('Data type of each column of Dataframe :')
print(dataTypeSeries)

In [ ]:
edges_days['Day'] = edges_days['Day'].astype(int)

In [ ]:
dataTypeSeries = edges_days.dtypes
print('Data type of each column of Dataframe :')
print(dataTypeSeries)

In [ ]:
edges_days.tail()

In [ ]:
edges_days.day = pd.to_numeric(edges_days.day, errors='coerce')

In [ ]:
G_days = nx.from_pandas_edgelist(edges_days, edge_attr = True, create_using=nx.DiGraph())

In [ ]:
G_days

In [ ]:
print(nx.info(G_days))

In [ ]:
edges_day = [(x,y,d) for x,y,d in G_days.edges.data() if d['day'] == 1]

In [ ]:
edges_day

## Creating features for separate days

In [ ]:

def computation_input_df(graph,day):
    # Creation of the input dataframe to return
    input_df = pd.DataFrame(columns = ['day', 'source', 'target', 'jaccard_coef', 'adamic_adar_index',
                                       'preferential_attachment_index', 'clustering_coefficient_score', 'weight'])
    input_df[['day', 'source', 'target']] = input_df[['day', 'source', 'target']].astype('int')
    input_df[['jaccard_coef', 'adamic_adar_index', 'preferential_attachment_index', 
              'clustering_coefficient_score', 'weight']] = input_df[['jaccard_coef', 'adamic_adar_index', 
              'preferential_attachment_index', 'clustering_coefficient_score', 'weight']].astype('float')
    
    # Creation of the subgraph for the day
    edges_day = [(x,y,d) for x,y,d in graph.edges.data() if d['day'] == day]
    print(edges_day)
    sub_graph = nx.DiGraph()
    sub_graph.add_edges_from(edges_day)
    sub_graph_undirected = nx.Graph()
    sub_graph_undirected.add_edges_from(edges_day)
    nodes = sorted(sub_graph.nodes)
    # Computation of Jaccard coefficient
    jac = nx.jaccard_coefficient(sub_graph_undirected, [(i,j) for i,j in itertools.product(nodes, nodes) 
                                                        if i!=j])
    jac_df = pd.DataFrame(list(jac), columns = ['source', 'target', 'jaccard_coef'])
    jac_df['day'] = day
    print('Jaccard: ok!')
    # Computation of Adamic and Adar index
    ad_ad = nx.adamic_adar_index(sub_graph_undirected, [(i,j) for i,j in itertools.product(nodes, nodes) 
                                                        if i!=j])
    ad_ad_df = pd.DataFrame(list(ad_ad), columns = ['source', 'target', 'adamic_adar_index'])
    print('Adamic and Adar: ok!')
    input_df_day = jac_df.merge(ad_ad_df)
    # Computation of Preferential Attachment
    pref_att = nx.preferential_attachment(sub_graph_undirected, [(i,j) for i,j in itertools.product(nodes, 
                                                                  nodes) if i!=j])
    pref_att_df = pd.DataFrame(list(pref_att), columns = ['source', 'target', 'preferential_attachment_index'])
    print('Preferential Attachment: ok!')
    
    
    input_df_day = input_df_day.merge(pref_att_df)
    # Clustering coefficient score
    
    
    #cluster_dict = nx.clustering(sub_graph)
    #ccs = [(i, j, (cluster_dict[i]*cluster_dict[j])) for i,j in itertools.product(nodes, nodes) if i!=j]
    #ccs_df = pd.DataFrame(list(ccs), columns = ['source', 'target', 'clustering_coefficient_score'])
    
    
    
    # Clustering coefficient score (on directed graph)
    cluster_dict = nx.clustering(sub_graph)
    cluster_dict_df = pd.DataFrame.from_dict(cluster_dict, orient='index', columns=['clustering_coefficient_score']).reset_index().rename(columns={'index':'User'})
    assert cluster_dict_df.shape[0] == len(nodes)
    #display(cluster_dict_df.head())
    # Add a column for clustering_coefficient_score_Source_User using the Clustering coefficient for the Source_User node
    input_df_day = input_df_day.merge(cluster_dict_df.rename(columns={'User': 'source', 
                                                               'clustering_coefficient_score': 'clustering_coefficient_score_Source_User'}),
                                on='source', how='left')
    # Add a column for clustering_coefficient_score_Target_User using the Clustering coefficient for the Target_User node
    input_df_day = input_df_day.merge(cluster_dict_df.rename(columns={'User': 'target', 
                                                               'clustering_coefficient_score': 'clustering_coefficient_score_Target_User'}),
                                on='target', how='left')
    
    
    
    print('Clustering coefficient: ok!')
    
    
      # Pagerank (on directed graph)
    pagerank_dict = nx.pagerank(sub_graph)
    pagerank_dict_df = pd.DataFrame.from_dict(pagerank_dict, orient='index', columns=['pagerank']).reset_index().rename(columns={'index':'User'})
    assert pagerank_dict_df.shape[0] == len(nodes)
     # Add a column for pagerank_Source_User using the Pagerank for the Source_User node
    input_df_day = input_df_day.merge(pagerank_dict_df.rename(columns={'User': 'source', 
                                                               'pagerank': 'pagerank_Source_User'}),
                                on='source', how='left')
    # Add a column for pagerank_Target_User using the Pagerank for the Target_User node
    input_df_day = input_df_day.merge(pagerank_dict_df.rename(columns={'User': 'target', 
                                                               'pagerank': 'pagerank_Target_User'}),
                                on='target', how='left')
    
    print('pagerank: ok!')
    #input_df_day = input_df_day.merge(ccs_df)
    # Add weights
    weights = pd.DataFrame([(n1, n2, sub_graph.edges[n1,n2]['weight']) 
                                 if (n1,n2) in sub_graph.edges else (n1, n2, 0)
                                 for n1 in nodes for n2 in nodes], 
                                 columns = ['source', 'target', 'weight'])
    print('Weights: ok!')
    input_df_day = input_df_day.merge(weights)
    input_df_day.to_csv('data/input_df_day_{}.csv'.format(str(day)), index = False)
    
    return input_df_day



In [ ]:
computation_input_df(G_days,4)

## Creating features for combined days

In [ ]:
def computation_input_df_combined(graph,start,end):
    print('Days {}-{}'.format(str(start),str(end)))
    # Creation of the input dataframe to return
    input_df = pd.DataFrame(columns = ['day', 'source', 'target', 'jaccard_coef', 'adamic_adar_index',
                                       'preferential_attachment_index', 'clustering_coefficient_score', 'weight'])
    input_df[['day', 'source', 'target']] = input_df[['day', 'source', 'target']].astype('int')
    input_df[['jaccard_coef', 'adamic_adar_index', 'preferential_attachment_index', 
              'clustering_coefficient_score', 'weight']] = input_df[['jaccard_coef', 'adamic_adar_index', 
              'preferential_attachment_index', 'clustering_coefficient_score', 'weight']].astype('float')
    
    # Creation of the subgraph for the day
    edges_day = [(x,y,d) for x,y,d in G_days.edges.data() if (d['day']>=start) & (d['day']<=end)]
    sub_graph = nx.DiGraph()
    sub_graph.add_edges_from(edges_day)
    sub_graph_undirected = nx.Graph()
    sub_graph_undirected.add_edges_from(edges_day)
    nodes = sorted(sub_graph.nodes)
    # Computation of Jaccard coefficient
    jac = nx.jaccard_coefficient(sub_graph_undirected, [(i,j) for i,j in itertools.product(nodes, nodes) 
                                                        if i!=j])
    jac_df = pd.DataFrame(list(jac), columns = ['source', 'target', 'jaccard_coef'])
    jac_df['day'] = '{}-{}'.format(str(start), str(end))
    print('Jaccard: ok!')
    # Computation of Adamic and Adar index
    ad_ad = nx.adamic_adar_index(sub_graph_undirected, [(i,j) for i,j in itertools.product(nodes, nodes) 
                                                        if i!=j])
    ad_ad_df = pd.DataFrame(list(ad_ad), columns = ['source', 'target', 'adamic_adar_index'])
    print('Adamic and Adar: ok!')
    input_df_day = jac_df.merge(ad_ad_df)
    # Computation of Preferential Attachment
    pref_att = nx.preferential_attachment(sub_graph_undirected, [(i,j) for i,j in itertools.product(nodes, 
                                                                  nodes) if i!=j])
    pref_att_df = pd.DataFrame(list(pref_att), columns = ['source', 'target', 'preferential_attachment_index'])
    print('Preferential Attachment: ok!')
    input_df_day = input_df_day.merge(pref_att_df)
    # Clustering coefficient score
    #cluster_dict = nx.clustering(sub_graph)
    #ccs = [(i, j, (cluster_dict[i]*cluster_dict[j])) for i,j in itertools.product(nodes, nodes) if i!=j]
    #ccs_df = pd.DataFrame(list(ccs), columns = ['source', 'target', 'clustering_coefficient_score'])
    #print('Clustering coefficient: ok!')
    #input_df_day = input_df_day.merge(ccs_df)
    
    cluster_dict = nx.clustering(sub_graph)
    cluster_dict_df = pd.DataFrame.from_dict(cluster_dict, orient='index', columns=['clustering_coefficient_score']).reset_index().rename(columns={'index':'User'})
    assert cluster_dict_df.shape[0] == len(nodes)
    #display(cluster_dict_df.head())
    # Add a column for clustering_coefficient_score_Source_User using the Clustering coefficient for the Source_User node
    input_df_day = input_df_day.merge(cluster_dict_df.rename(columns={'User': 'source', 
                                                               'clustering_coefficient_score': 'clustering_coefficient_score_Source_User'}),
                                on='source', how='left')
    # Add a column for clustering_coefficient_score_Target_User using the Clustering coefficient for the Target_User node
    input_df_day = input_df_day.merge(cluster_dict_df.rename(columns={'User': 'target', 
                                                               'clustering_coefficient_score': 'clustering_coefficient_score_Target_User'}),
                                on='target', how='left')
    
    
    
    print('Clustering coefficient: ok!')
    
    
      # Pagerank 
    pagerank_dict = nx.pagerank(sub_graph)
    pagerank_dict_df = pd.DataFrame.from_dict(pagerank_dict, orient='index', columns=['pagerank']).reset_index().rename(columns={'index':'User'})
    assert pagerank_dict_df.shape[0] == len(nodes)
     # Add a column for pagerank_Source_User using the Pagerank for the Source_User node
    input_df_day = input_df_day.merge(pagerank_dict_df.rename(columns={'User': 'source', 
                                                               'pagerank': 'pagerank_Source_User'}),
                                on='source', how='left')
    # Add a column for pagerank_Target_User using the Pagerank for the Target_User node
    input_df_day = input_df_day.merge(pagerank_dict_df.rename(columns={'User': 'target', 
                                                               'pagerank': 'pagerank_Target_User'}),
                                on='target', how='left')
    
    print('pagerank: ok!')
    
    
    
    
    # Add weights
    weights = pd.DataFrame([(n1, n2, sub_graph.edges[n1,n2]['weight']) 
                                 if (n1,n2) in sub_graph.edges else (n1, n2, 0)
                                 for n1 in nodes for n2 in nodes], 
                                 columns = ['source', 'target', 'weight'])
    print('Weights: ok!')
    input_df_day = input_df_day.merge(weights)
    input_df_day.to_csv('input_df_day_{}_{}.csv'.format(str(start),str(end)), 
                        index = False)
    

In [ ]:
computation_input_df_combined(G_days,1,3)

In [ ]:
input_df_day = input_df_day.merge(weights)
input_df_day.to_csv('input_df_day_{}_{}.csv'.format(str(1),str(1)), 
                        index = False)
    